## 전이학습 참고
- 분류기 끝 부분만 수정
- 분류기 자체를 새로 ~ '동결'
- high level에서 특징추출 layer를 수정
- 관련 논문 찾아봐서 참고

### 사용 데이터 셋
- Kaggle Image Classification data set : https://www.kaggle.com/puneet6060/intel-image-classification
- 150x150, 6 classes (buildings, forest, glacier, mountain, sea, street)
- seg_train / seg_test : 학습셋, 테스트셋 // seg_pred : 라벨링 안 되어있는 이미지 모음 ~ 모델 최종 예측값 확인

## VGGNet
- keras api
- kernel 3x3 고정
- layer 깊이에 따른 모델 성능 확인 등등 조사 내용 추가

In [11]:
# 기존 VGG16 비교 - keras 책 p.202
import tensorflow as tf
from tensorflow.keras.applications import VGG16
# 함수형 api로 재구현
from tensorflow.keras import layers
from tensorflow.keras import Input
from tensorflow.keras.models import Model
# 사용 데이터셋 읽기
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import callbacks

In [2]:
# 기존 VGG 구조 확인
ori_vgg16 = VGG16(weights='imagenet', include_top=True)
ori_vgg16.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [3]:
sized_vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(150,150,3))
sized_vgg16.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

In [ ]:
# 함수형 api로 재구성 - acc 0.17 정도 나오는 코드
image_input = Input(shape=(150,150,3)) # input_layer / 150x150xRGB
# block1
x = layers.Conv2D(64, 3, activation='relu', padding='same')(image_input) # block1_conv1 : Conv2D layer / 64_filter, kenel 3x3, 이미지 크기 변하지 않도록 same padding
x = layers.Conv2D(64, 3, activation='relu', padding='same')(x) # block1_conv2 : Conv2D layer / 64_filter, kenel 3x3, 이미지 크기 변하지 않도록 same padding
x = layers.MaxPooling2D(2, strides=2)(x) # block1_pool : MaxPooling2D layer / pool_size=2, stride=2 ~ 이미지 크기 (75,75)로 줄어듦음
# block2
x = layers.Conv2D(128, 3, activation='relu', padding='same')(x) # block2_conv1 : Conv2D layer / 128_filter, kenel 3x3, 이미지 크기 변하지 않도록 same padding
x = layers.Conv2D(128, 3, activation='relu', padding='same')(x) # block2_conv2 : Conv2D layer / 128_filter, kenel 3x3, 이미지 크기 변하지 않도록 same padding
x = layers.MaxPooling2D(2, strides=2)(x) # block2_pool : MaxPooling2D layer / pool_size=2, stride=2 ~ 이미지 크기 (37,37)로 줄어듦음
# block3
x = layers.Conv2D(256, 3, activation='relu', padding='same')(x) # block3_conv1 : Conv2D layer / 256_filter, kenel 3x3, 이미지 크기 변하지 않도록 same padding
x = layers.Conv2D(256, 3, activation='relu', padding='same')(x) # block3_conv2 : Conv2D layer / 256_filter, kenel 3x3, 이미지 크기 변하지 않도록 same padding
x = layers.Conv2D(256, 3, activation='relu', padding='same')(x) # block3_conv3 : Conv2D layer / 256_filter, kenel 3x3, 이미지 크기 변하지 않도록 same padding
x = layers.MaxPooling2D(2, strides=2)(x) # block3_pool : MaxPooling2D layer / pool_size=2, stride=2 ~ 이미지 크기 (18,18)로 줄어듦음
# block4 // 210122 : 필터 수 512에서 128로 변경
x = layers.Conv2D(128, 3, activation='relu', padding='same')(x) # block4_conv1 : Conv2D layer / 512_filter, kenel 3x3, 이미지 크기 변하지 않도록 same padding
x = layers.Conv2D(128, 3, activation='relu', padding='same')(x) # block4_conv2 : Conv2D layer / 512_filter, kenel 3x3, 이미지 크기 변하지 않도록 same padding
x = layers.Conv2D(128, 3, activation='relu', padding='same')(x) # block4_conv3 : Conv2D layer / 512_filter, kenel 3x3, 이미지 크기 변하지 않도록 same padding
x = layers.MaxPooling2D(2, strides=2)(x) # block4_pool : MaxPooling2D layer / pool_size=2, stride=2 ~ 이미지 크기 (9,9)로 줄어듦음
# block5 // 210122 : 필터 수 512에서 64로 변경
x = layers.Conv2D(64, 3, activation='relu', padding='same')(x) # block5_conv1 : Conv2D layer / 512_filter, kenel 3x3, 이미지 크기 변하지 않도록 same padding
x = layers.Conv2D(64, 3, activation='relu', padding='same')(x) # block5_conv2 : Conv2D layer / 512_filter, kenel 3x3, 이미지 크기 변하지 않도록 same padding
x = layers.Conv2D(64, 3, activation='relu', padding='same')(x) # block5_conv3 : Conv2D layer / 512_filter, kenel 3x3, 이미지 크기 변하지 않도록 same padding
x = layers.MaxPooling2D(2, strides=2)(x) # block5_pool : MaxPooling2D layer / pool_size=2, stride=2 ~ 이미지 크기 (4,4)로 줄어듦음
# fc 층 연결
x = layers.Flatten()(x) # fc층 이전 flatten
x = layers.Dense(1024, activation='relu')(x) # (4,4,512) -> 8192 => 4096 (절반) // 210122 : 1024로 변경
x = layers.Dense(512, activation='relu')(x) # 4096 => 2048 (절반) // 210122 : 512로 변경
output = layers.Dense(6, activation='softmax')(x) # 최종 클래스 6개

In [5]:
model = Model(image_input, output)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 150, 150, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 150, 150, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 75, 75, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 75, 75, 128)       73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 75, 75, 128)       147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 37, 37, 128)       0     

In [ ]:
# 데이터셋 불러오기
## validation_split?
train_datagen = ImageDataGenerator(rescale = 1./255)
train_generator = train_datagen.flow_from_directory('/content/gdrive/MyDrive/vgg_practice_set/seg_train/seg_train', # 파일 위치 확인
                                                    target_size=(150,150),
                                                    batch_size = 5,
                                                    class_mode='categorical')

test_datagen = ImageDataGenerator(rescale = 1./255)
test_generator = test_datagen.flow_from_directory('/content/gdrive/MyDrive/vgg_practice_set/seg_test/seg_test', # 파일 위치 확인
                                                  target_size=(150,150),
                                                  batch_size=5,
                                                  class_mode='categorical')

In [ ]:
# callback 및 compile 설정
callback_list = [callbacks.EarlyStopping(monitor='val_loss', patience=10),
                 callbacks.ModelCheckpoint(filepath='./modelcheckpoint',monitor='val_loss',save_best_only=True)]

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit_generator(train_generator,
                              epochs = 200, # 30 -> 200 으로 증가
                              steps_per_epoch = 2806, # 14030 /5 = 2806
                              validation_data=test_generator,
                              callbacks = callback_list,
                              validation_steps=600 # 3000 / 5 = 600
                              )

In [ ]:
model.save('colab_vgg_test0122_kaggleCode.h5')

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
y_vloss = history.history['val_loss']
y_loss = history.history['loss']

In [ ]:
x_len = np.arange(len(y_acc))
plt.plot(x_len, acc, ".", c="red", label='trainset_acc')
plt.plot(x_len, val_acc, ".", c="lightcoral", label='testset_acc' )
plt.plot(x_len, y_vloss, ".", c="cornflowerblue", label='testset_loss')
plt.plot(x_len, y_loss, ".", c="blue", label='trainset_loss')
plt.show()

- Kaggle 코드 참고 : https://www.kaggle.com/trolukovich/vgg-like-87-accuracy-keras
- kernel_initializer, batch_normalrization, kernel_regularization, momentum
- Batch Normalization : 신경망 입력값을 평균 0, 분산 1로 정규화
- batch normal doc 참고 : https://keras.io/api/layers/normalization_layers/batch_normalization/
- 블로그 추가 참고 : https://eremo2002.tistory.com/55 / https://shuuki4.wordpress.com/2016/01/13/batch-normalization-%EC%84%A4%EB%AA%85-%EB%B0%8F-%EA%B5%AC%ED%98%84/

## ResNet
- keras api
- 참고 블로그 : https://eremo2002.tistory.com/76

In [5]:
# 기존 ResNet 구조 확인
from tensorflow.keras.applications.resnet import ResNet50

In [ ]:
ori_res50 = ResNet50(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)
#AttributeError: 'str' object has no attribute 'decode' 오류 나면서 원본 구조 확인 불가?
ori_res50.summary()

In [ ]:
# 참고 : https://eremo2002.tistory.com/76
res_input = Input(shape=(150,150,3)) # input_layer / 150x150xRGB
# conv1: Conv2D layer / 64_filter, kenel 7x7, 이미지 크기 변하지 않도록 same padding? ~ output size 보고 valid padding으로 변경 필요
x = layers.Conv2D(filters=64, kernel_size=7, strides=2, activation='relu', padding='same')(image_input)
x = layers.MaxPooling2D(3, strides=2)(x) # block1_pool : MaxPooling2D layer / pool_size=3, stride=2 ~ 이미지 크기 (75,75)로 줄어듦음
# conv2_x

res_output = layers.Dense(6, activation='softmax')(x) # 최종 클래스 6개

In [ ]:
### 복붙용
x = layers.Conv2D(128, 3, activation='relu', padding='same')(x) # block2_conv1 : Conv2D layer / 128_filter, kenel 3x3, 이미지 크기 변하지 않도록 same padding
x = layers.Conv2D(128, 3, activation='relu', padding='same')(x) # block2_conv2 : Conv2D layer / 128_filter, kenel 3x3, 이미지 크기 변하지 않도록 same padding
x = layers.MaxPooling2D(2, strides=2)(x) # block2_pool : MaxPooling2D layer / pool_size=2, stride=2 ~ 이미지 크기 (37,37)로 줄어듦음
# block3
x = layers.Conv2D(256, 3, activation='relu', padding='same')(x) # block3_conv1 : Conv2D layer / 256_filter, kenel 3x3, 이미지 크기 변하지 않도록 same padding
x = layers.Conv2D(256, 3, activation='relu', padding='same')(x) # block3_conv2 : Conv2D layer / 256_filter, kenel 3x3, 이미지 크기 변하지 않도록 same padding
x = layers.Conv2D(256, 3, activation='relu', padding='same')(x) # block3_conv3 : Conv2D layer / 256_filter, kenel 3x3, 이미지 크기 변하지 않도록 same padding
x = layers.MaxPooling2D(2, strides=2)(x) # block3_pool : MaxPooling2D layer / pool_size=2, stride=2 ~ 이미지 크기 (18,18)로 줄어듦음
# block4
x = layers.Conv2D(512, 3, activation='relu', padding='same')(x) # block4_conv1 : Conv2D layer / 512_filter, kenel 3x3, 이미지 크기 변하지 않도록 same padding
x = layers.Conv2D(512, 3, activation='relu', padding='same')(x) # block4_conv2 : Conv2D layer / 512_filter, kenel 3x3, 이미지 크기 변하지 않도록 same padding
x = layers.Conv2D(512, 3, activation='relu', padding='same')(x) # block4_conv3 : Conv2D layer / 512_filter, kenel 3x3, 이미지 크기 변하지 않도록 same padding
x = layers.MaxPooling2D(2, strides=2)(x) # block4_pool : MaxPooling2D layer / pool_size=2, stride=2 ~ 이미지 크기 (9,9)로 줄어듦음
# block5
x = layers.Conv2D(512, 3, activation='relu', padding='same')(x) # block5_conv1 : Conv2D layer / 512_filter, kenel 3x3, 이미지 크기 변하지 않도록 same padding
x = layers.Conv2D(512, 3, activation='relu', padding='same')(x) # block5_conv2 : Conv2D layer / 512_filter, kenel 3x3, 이미지 크기 변하지 않도록 same padding
x = layers.Conv2D(512, 3, activation='relu', padding='same')(x) # block5_conv3 : Conv2D layer / 512_filter, kenel 3x3, 이미지 크기 변하지 않도록 same padding
x = layers.MaxPooling2D(2, strides=2)(x) # block5_pool : MaxPooling2D layer / pool_size=2, stride=2 ~ 이미지 크기 (4,4)로 줄어듦음
# fc 층 연결
x = layers.Flatten()(x) # fc층 이전 flatten
x = layers.Dense(4096, activation='relu')(x) # (4,4,512) -> 8192 => 4096 (절반)
x = layers.Dense(2048, activation='relu')(x) # 4096 => 2048 (절반)
output = layers.Dense(6, activation='softmax')(x) # 최종 클래스 6개

In [ ]:
res_model = Model(res_input, output)
res_model.summary()